In [1]:
%load_ext dockermagic

# Hadoop - multi-node cluster setup 
![Hadoop](https://hadoop.apache.org/elephant.png)

## Create Hadoop base image

### Create docker container

- Ubuntu 18.04 (https://ubuntu.com/)
- Docker (https://www.docker.com/)
    - container based virtualization

In [2]:
%%bash

docker run -d -t --rm --name hadoopimg -h hadoopimg ubuntu:18.04

docker ps

Unable to find image 'ubuntu:18.04' locally
18.04: Pulling from library/ubuntu
13f679bb266a: Pulling fs layer
13f679bb266a: Verifying Checksum
13f679bb266a: Download complete
13f679bb266a: Pull complete
Digest: sha256:8aa9c2798215f99544d1ce7439ea9c3a6dfd82de607da1cec3a8a2fae005931b
Status: Downloaded newer image for ubuntu:18.04


a0587614fdd18894f9ede098f32ba377ce4e7e36ee51438c3e02a2165284741d
CONTAINER ID   IMAGE          COMMAND       CREATED                  STATUS                  PORTS     NAMES
a0587614fdd1   ubuntu:18.04   "/bin/bash"   Less than a second ago   Up Less than a second             hadoopimg


### Install Dependencies

- Java 8 (OpenJDK) - https://cwiki.apache.org/confluence/display/HADOOP/Hadoop+Java+Versions
- Other packages: ssh pdsh wget apt-utils

In [3]:
%%dockerexec hadoopimg

# Update package list
apt -qq update > /install.log 2>&1

# Install Hadoop dependencies
apt -qq -f -y install openjdk-8-jdk ssh pdsh >> /install.log 2>&1

# Install other dependencies
apt -qq -f -y install vim wget apt-utils python3 python3-pip \
    ipython3 less unzip sudo net-tools >> /install.log 2>&1

### Install Hadoop

- http://hadoop.apache.org/
- Version 3.2.4
- Base directory: /opt
- User/group: hadoop/hadoop
- Package with binaries (version 3.2.4): https://hadoop.apache.org/releases.html

In [4]:
%%dockerexec hadoopimg

# Enable rwx for all on /opt
chmod 777 /opt

# Create user/group hadoop
useradd -m -U -s /bin/bash hadoop

# Enable sudo for hadoop
sed -i "\$ahadoop  ALL=(ALL) NOPASSWD:ALL" /etc/sudoers

In [11]:
%%bash

HADOOPVERSION=hadoop-3.2.4

# Download package
cd ../pkgs
wget -q -c https://dlcdn.apache.org/hadoop/common/$HADOOPVERSION/$HADOOPVERSION.tar.gz

# Copy installation package to container
docker cp $HADOOPVERSION.tar.gz hadoopimg:/opt

In [12]:
%%dockerexec -u hadoop hadoopimg

HADOOPVERSION=hadoop-3.2.4

# Modify user/group permissions and unpack file
sudo chown hadoop:hadoop /opt/$HADOOPVERSION.tar.gz
tar -zxf /opt/$HADOOPVERSION.tar.gz -C /opt
rm /opt/$HADOOPVERSION.tar.gz

# Create link
ln -s /opt/$HADOOPVERSION /opt/hadoop

### Configure environment variables

- Create file /opt/envvars.sh with environment variables

In [13]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/envvars.sh << EOF
export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh

export HADOOP_HOME=/opt/hadoop
export HADOOP_COMMON_HOME=\${HADOOP_HOME}
export HADOOP_CONF_DIR=\${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=\${HADOOP_HOME}
export HADOOP_MAPRED_HOME=\${HADOOP_HOME}
export HADOOP_YARN_HOME=\${HADOOP_HOME}

export PATH=\${PATH}:\${HADOOP_HOME}/bin:\${HADOOP_HOME}/sbin     

EOF

cat /opt/envvars.sh

export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh

export HADOOP_HOME=/opt/hadoop
export HADOOP_COMMON_HOME=${HADOOP_HOME}
export HADOOP_CONF_DIR=${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=${HADOOP_HOME}
export HADOOP_MAPRED_HOME=${HADOOP_HOME}
export HADOOP_YARN_HOME=${HADOOP_HOME}

export PATH=${PATH}:${HADOOP_HOME}/bin:${HADOOP_HOME}/sbin     



### Configure passwordless ssh

In [14]:
%%dockerexec -u hadoop hadoopimg

# Disable host key checking
sudo tee -a /etc/ssh/ssh_config << EOF
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
EOF

# Create ssh key
ssh-keygen -q -t rsa -P "" -f ~/.ssh/id_rsa

# Copy public key to authorized_keys file
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys

    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null


### Hadoop configuration files

- Hadoop configuration files location: \$HADOOP\_HOME\/etc\/hadoop
- All cluster nodes contain the same files

#### hadoop-env.sh

- Definition of environment variables used by Hadoop processes

In [15]:
%%dockerexec -u hadoop hadoopimg

cat >> /opt/hadoop/etc/hadoop/hadoop-env.sh << EOF
export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
EOF

#### core-site.xml

- Hadoop main configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-common/core-default.xml

In [16]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/core-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>fs.defaultFS</name>
    <value>hdfs://hadoop:9000</value>
</property>

<property>
    <name>hadoop.proxyuser.hadoop.groups</name>
    <value>*</value>
</property>

<property>
    <name>hadoop.proxyuser.hadoop.hosts</name>
    <value>*</value>
</property>

</configuration>
EOF

#### hdfs-site.xml

- HDFS configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml

In [17]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/hdfs-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>dfs.namenode.name.dir</name>
    <value>/opt/hadoop/data/nameNode</value>
</property>

<property>
    <name>dfs.datanode.data.dir</name>
    <value>/opt/hadoop/data/dataNode</value>
</property>

<property>
    <name>dfs.replication</name>
    <value>2</value>
</property>

<property>
    <name>dfs.blocksize</name>
    <value>33554432</value>
</property>

<property>
    <name>dfs.hosts.exclude</name>
    <value>/opt/hadoop/etc/hadoop/dfs.exclude</value>
</property>

<property>
    <name>dfs.namenode.heartbeat.recheck-interval</name>
    <value>10000</value>
</property>

</configuration>

EOF

#### yarn-site.xml

- YARN configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-yarn/hadoop-yarn-common/yarn-default.xml

In [18]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/yarn-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<configuration>

<property>
    <name>yarn.resourcemanager.hostname</name>
    <value>hadoop</value>
</property>

<property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
</property>

<property>
    <name>yarn.nodemanager.resource.memory-mb</name>
    <value>1536</value>
</property>

<property>
    <name>yarn.scheduler.maximum-allocation-mb</name>
    <value>1536</value>
</property>

<property>
    <name>yarn.scheduler.minimum-allocation-mb</name>
    <value>128</value>
</property>

<property>
    <name>yarn.timeline-service.enabled</name>
    <value>true</value>
</property>

<property>
    <name>yarn.timeline-service.hostname</name>
    <value>hadoop</value>
</property>

<property>
    <name>yarn.system-metrics-publisher.enabled</name>
    <value>true</value>
</property>

<property>
    <name>yarn.log-aggregation-enable</name>
    <value>true</value>
</property>

<property>
    <name>yarn.nm.liveness-monitor.expiry-interval-ms</name>
    <value>10000</value>
</property>

</configuration>
EOF

#### mapred-site.xml

- MapReduce configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-mapreduce-client/hadoop-mapreduce-client-core/mapred-default.xml

In [19]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/mapred-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
</property>

<property>
    <name>mapreduce.application.classpath</name>
    <value>/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/mapreduce/lib/*</value>
</property>

<property>
    <name>yarn.app.mapreduce.am.resource.mb</name>
    <value>512</value>
</property>

<property>
    <name>mapreduce.map.memory.mb</name>
    <value>256</value>
</property>

<property>
    <name>mapreduce.reduce.memory.mb</name>
    <value>256</value>
</property>

</configuration>
EOF

#### workers

- List of worker nodes (NodeManager and DataNode)

In [20]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/workers << EOF
hadoop1
hadoop2
hadoop3
EOF

### Commit base image

In [21]:
%%bash

# Create hadoopimg image based on hadoop container
docker commit hadoopimg hadoopimg

# Stop base container
docker stop hadoopimg

sha256:1b09822962966bb2addff9aa8938be8f3883e084e063c16c5e61df105b0acf18
hadoopimg


## Create cluster

### Run nodes

In [22]:
%%bash

# MASTER

# Ports
# 9870 - Namenode
# 9868 - Secondary Namenode
# 8088 - ResourceManager
# 19888 - MapReduce Job History
# 8188 - Timeline Service
# 4040 - Spark Application UI
# 8080 - Jupyter

cd ..
docker run -d -t --memory 4g --memory-swap 4g --rm --name hadoop -h hadoop -u hadoop \
    -v "$(pwd)"/pkgs:/opt/pkgs -v "$(pwd)"/notebooks:/opt/notebooks -v "$(pwd)"/datasets:/opt/datasets \
    -p 9870:9870 -p 9868:9868 -p 8088:8088 -p 19888:19888 -p 8188:8188 -p 4040:4040 -p 8080:8080 hadoopimg

# WORKERS

# Ports
# 9864 - DataNode WebUI
# 8042 - NodeManager WebUI

# Hadoop1
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop1 -h hadoop1 -u hadoop \
    -p 9864:9864 -p 8042:8042 hadoopimg
# Hadoop2
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop2 -h hadoop2 -u hadoop \
    -p 9865:9864 -p 8043:8042  hadoopimg
# Hadoop3
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop3 -h hadoop3 -u hadoop \
    -p 9866:9864 -p 8044:8042  hadoopimg

docker ps

86319dfbc48faee5578f7830a8d09232b9262daab530401b5aae5820a915b070
c9eb315ea92a21fd4a0abea6bd0c08ee3897165e9cfe96dddfccf1a876e1038d
ed14fd88984e3bce616dcb0e7944087d7e72bd0119e3195902d6db02b4a80bd7
26dcbc50eb9f50ed97b5c917687124a77c2f603e5456eba615a3fbcc06108406
CONTAINER ID   IMAGE       COMMAND       CREATED                  STATUS                  PORTS                                                                                                                                                                      NAMES
26dcbc50eb9f   hadoopimg   "/bin/bash"   Less than a second ago   Up Less than a second   0.0.0.0:8044->8042/tcp, 0.0.0.0:9866->9864/tcp                                                                                                                             hadoop3
ed14fd88984e   hadoopimg   "/bin/bash"   Less than a second ago   Up Less than a second   0.0.0.0:8043->8042/tcp, 0.0.0.0:9865->9864/tcp                                                                    

### Configure hosts file on all nodes

- /etc/hosts

In [23]:
%%bash

# Get IPs
M=$(docker inspect hadoop | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H1=$(docker inspect hadoop1 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H2=$(docker inspect hadoop2 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H3=$(docker inspect hadoop3 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)

# Create hosts file
cat > hosts << EOF  
$M hadoop
$H1 hadoop1
$H2 hadoop2
$H3 hadoop3
EOF

cat hosts

# Copy to all nodes
docker exec -i -u root hadoop sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop1 sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop2 sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop3 sh -c 'cat >> /etc/hosts' < hosts

# Remove local file
rm hosts

172.17.0.2 hadoop
172.17.0.3 hadoop1
172.17.0.4 hadoop2
172.17.0.5 hadoop3


### Start ssh server on all nodes

In [24]:
%%bash

for HOST in hadoop hadoop1 hadoop2 hadoop3
do
    echo $HOST
    docker exec -u root $HOST service ssh restart
    docker exec -u root $HOST service ssh status
done

hadoop
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop1
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop2
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop3
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running


## Format HDFS on Namenode

In [26]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs namenode -format -force -nonInteractive

2023-03-27 13:38:14,472 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = hadoop/172.17.0.2
STARTUP_MSG:   args = [-format, -force, -nonInteractive]
STARTUP_MSG:   version = 3.2.4
STARTUP_MSG:   classpath = /opt/hadoop/etc/hadoop:/opt/hadoop/share/hadoop/common/lib/jetty-servlet-9.4.43.v20210629.jar:/opt/hadoop/share/hadoop/common/lib/stax2-api-4.2.1.jar:/opt/hadoop/share/hadoop/common/lib/kerby-pkix-1.0.1.jar:/opt/hadoop/share/hadoop/common/lib/metrics-core-3.2.4.jar:/opt/hadoop/share/hadoop/common/lib/jetty-util-9.4.43.v20210629.jar:/opt/hadoop/share/hadoop/common/lib/httpcore-4.4.13.jar:/opt/hadoop/share/hadoop/common/lib/jsr311-api-1.1.1.jar:/opt/hadoop/share/hadoop/common/lib/spotbugs-annotations-3.1.9.jar:/opt/hadoop/share/hadoop/common/lib/zookeeper-3.4.14.jar:/opt/hadoop/share/hadoop/common/lib/jersey-servlet-1.19.jar:/opt/hadoop/share/hadoop/common/lib/jersey-server-1.19.jar:

## Start Hadoop daemons

- manual execution: ```hdfs --daemon start (namenode|datanode)``` and ```yarn --daemon start (resourcemanager|nodemanager)```
- auxilliary scripts to run all processes on the cluster: start-dfs.sh (HDFS) and start-yarn.sh (YARN)
- some services still need to be executed manually (timelineserver, historyserver)

In [28]:
%%dockerexec hadoop

source /opt/envvars.sh

# HDFS
start-dfs.sh

# YARN
start-yarn.sh

# timelineserver
yarn --daemon start timelineserver

# historyserver
mapred --daemon start historyserver

Starting namenodes on [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
hadoop: namenode is running as process 341.  Stop it first and ensure /tmp/hadoop-hadoop-namenode.pid file is empty before retry.
pdsh@hadoop: hadoop: ssh exited with exit code 1
Starting datanodes
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hosts.
hadoop2: WARNING: /opt/hadoop-3.2.4/logs does not exist. Creating.
hadoop3: WARNING: /opt/hadoop-3.2.4/logs does not exist. Creating.
hadoop1: WARNING: /opt/hadoop-3.2.4/logs does not exist. Creating.
Starting secondary namenodes [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
hadoop: secondarynamenode is running as process 580.  Stop it first and

In [29]:
%%bash

# Listing all processes
for HOST in hadoop hadoop1 hadoop2 hadoop3; do
    echo $HOST
    docker exec $HOST jps
done

hadoop
2082 Jps
580 SecondaryNameNode
341 NameNode
822 ResourceManager
1255 JobHistoryServer
hadoop1
418 Jps
293 NodeManager
172 DataNode
hadoop2
163 DataNode
409 Jps
284 NodeManager
hadoop3
165 DataNode
411 Jps
286 NodeManager


## Create HDFS directories

In [30]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -mkdir -p /user/hadoop
hdfs dfs -chown hadoop:hadoop /user/hadoop
hdfs dfs -mkdir /tmp
hdfs dfs -chmod 777 /tmp

2023-03-27 13:41:51,667 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-03-27 13:41:52,264 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-03-27 13:41:52,849 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
mkdir: `/tmp': File exists
2023-03-27 13:41:53,421 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Install and run Jupyter on master node

In [32]:
%%dockerexec hadoop

# pip3 -q install notebook
pip3 -q install pip
pip3 -q install jupyterlab

IP=$(ifconfig eth0 | grep inet | awk '{ print $2 }')

cd /opt

export SHELL=/bin/bash
# nohup /home/hadoop/.local/bin/jupyter-notebook --ip=$IP --port=8080 --NotebookApp.token='' --NotebookApp.password='' --NotebookApp.notebook_dir='/' --no-browser &
nohup /home/hadoop/.local/bin/jupyter-lab --ip=$IP --port=8080 --notebook-dir='/' --ServerApp.token='' --ServerApp.password='' --no-browser &

echo $! > /tmp/jupyter.pid

# To kill
# kill $(cat /tmp/jupyter.pid)

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
[I 2023-03-27 13:44:35.441 ServerApp] jupyterlab | extension was successfully linked.
[I 2023-03-27 13:44:35.445 ServerApp] Writing Jupyter server cookie secret to /home/hadoop/.local/share/jupyter/runtime/jupyter_cookie_secret
[I 2023-03-27 13:44:35.538 ServerApp] nbclassic | extension was successfully linked.
[W 2023-03-27 13:44:35.546 ServerApp] All authentication is disabled.  Anyone who can connect to this server will be able to run code.
[I 2023-03-27 13:44:35.549 ServerApp] nbclassic | extension was successfully loaded.
[I 2023-03-27 13:44:35.549 LabApp] JupyterLab extension loaded from /home/hadoop/.local/lib/pytho

## Access web interfaces

- Jupyterlab
    - http://localhost:8080
- Master - hadoop
    - Resource Manager: http://localhost:8088
    - NameNode: http://localhost:9870
    - Secondary NameNode: http://localhost:9868
    - MapReduce Job History: http://localhost:19888
    - Timeline Service: http://localhost:8188
- Workers
    - hadoop1
        - NodeManager: http://localhost:8042
        - DataNode: http://localhost:9864
    - hadoop2
        - NodeManager: http://localhost:8043
        - DataNode: http://localhost:9865
    - hadoop3
        - NodeManager: http://localhost:8044
        - DataNode: http://localhost:9866

## Run mapreduce Pi example

In [35]:
%%dockerexec hadoop

source /opt/envvars.sh
cd /opt/hadoop/share/hadoop/mapreduce

hadoop jar ./hadoop-mapreduce-examples-3.2.4.jar pi 6 10000

Number of Maps  = 6
Samples per Map = 10000
2023-03-27 13:48:10,875 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Wrote input for Map #0
Wrote input for Map #1
Wrote input for Map #2
Wrote input for Map #3
Wrote input for Map #4
Wrote input for Map #5
Starting Job
2023-03-27 13:48:11,787 INFO client.RMProxy: Connecting to ResourceManager at hadoop/172.17.0.2:8032
2023-03-27 13:48:11,866 INFO client.AHSProxy: Connecting to Application History server at hadoop/172.17.0.2:10200
2023-03-27 13:48:11,916 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1679924314464_0002
2023-03-27 13:48:11,967 INFO input.FileInputFormat: Total input files to process : 6
2023-03-27 13:48:11,995 INFO mapreduce.JobSubmitter: number of splits:6
2023-03-27 13:48:12,047 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1679924314464_0002
2023-03-27 13:48:

# SHUTDOWN PROCEDURE

## Stop Jupyter

In [36]:
%%dockerexec hadoop

kill $(cat /tmp/jupyter.pid)

## Stop Hadoop daemons

In [37]:
%%dockerexec hadoop

source /opt/envvars.sh

stop-dfs.sh
stop-yarn.sh
yarn --daemon stop timelineserver
mapred --daemon stop historyserver

Stopping namenodes on [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
Stopping datanodes
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hosts.
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
Stopping secondary namenodes [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
2023-03-27 13:53:53,388 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping nodemanagers
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hos

## Stop Docker containers

In [38]:
%%bash

for HOST in hadoop hadoop1 hadoop2 hadoop3; do
    docker stop $HOST
done

docker ps

hadoop
hadoop1
hadoop2
hadoop3
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
